In [2]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
#import seaborn as sns
#import matplotlib.pyplot as plt
import json
from bson import json_util
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,functions as F

In [14]:
#global variables

g_appname="nest_spark_get_mongodb"
g_master="local[2]"
g_memory="30g"

#Local
g_input="mongodb://127.0.0.1/backend_production.properties"

#Production
#g_input="mongodb://3.210.155.32/backend_production.properties"


In [15]:

### set Spark session
my_spark = SparkSession \
     .builder \
     .appName(g_appname) \
     .master(g_master) \
     .config('spark.driver.memory', g_memory)\
     .config("spark.mongodb.input.uri", g_input ) \
     .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.0')\
     .getOrCreate()


     #.config("spark.mongodb.output.uri", "mongodb://3.210.155.32/backend_production.properties") \
        

In [16]:
# Set SQL Context
sqlContext=SQLContext(my_spark)


In [17]:
# Read data
df = my_spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","backend_production")\
    .option("collection", "properties")\
    .load()

In [19]:
#from nest_db_properties_spark import get_mongo_data_spark
#df=get_mongo_data_spark()


In [20]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: null (nullable = true)
 |-- basement: boolean (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:Ful

In [23]:
df.dtypes

[('_id', 'struct<oid:string>'),
 ('address_number', 'string'),
 ('address_street', 'string'),
 ('agent_avatar_url', 'null'),
 ('agent_email', 'string'),
 ('agent_external_id', 'string'),
 ('agent_fname', 'string'),
 ('agent_id', 'string'),
 ('agent_key', 'string'),
 ('agent_lname', 'string'),
 ('agent_name', 'string'),
 ('agent_phone', 'string'),
 ('architecture_style', 'string'),
 ('available_showing', 'null'),
 ('basement', 'boolean'),
 ('bathrooms', 'double'),
 ('beds', 'int'),
 ('brokerage',
  'struct<address:struct<commons:City:string,commons:Country:string,commons:FullStreetAddress:string,commons:PostalCode:string,commons:StateOrProvince:string,commons:UnitNumber:string,commons:address-preference-order:string,commons:preference-order:string>,logo_url:string,name:string,phone:string,website_url:string>'),
 ('brokerage_email', 'string'),
 ('brokerage_external_id', 'string'),
 ('brokerage_logo_url', 'string'),
 ('brokerage_name', 'string'),
 ('brokerage_phone', 'string'),
 ('brokera

In [32]:
df.schema['year_built'].dataType

IntegerType

In [33]:
df.schema['water_heater'].dataType

StringType

In [48]:
df.schema['total_sqft'].dataType

DoubleType

In [35]:
df.schema['stories'].dataType

NullType

In [36]:
df.schema['pool'].dataType

BooleanType

In [113]:
from pyspark.sql.types import *
for i in df.columns:
    df= df.withColumn(i, df[i].cast(StringType()))
    

In [108]:
'''
from pyspark.sql.types import *
for i in df.columns:
    #df= df.withColumn(i, df[i].cast(StringType()))
    #print(df.schema[i].dataType)
    b=df.schema['stories'].dataType
    
    if ((df.schema[i].dataType == df.schema['stories'].dataType) | (df.schema[i].dataType == df.schema['total_sqft'].dataType)\
         | (df.schema[i].dataType == df.schema['basement'].dataType) | (df.schema[i].dataType == df.schema['brokerage'].dataType)
         | (df.schema[i].dataType == df.schema['external_type'].dataType) 
       ):
        print(df.schema[i].dataType)
        df= df.withColumn(i, df[i].cast(StringType()))
        
'''




StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StructType(List(StructField(symbol,StringType,true)))
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType


In [114]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: string (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- brokerage: string (nullable = true)
 |-- brokerage_email: string (nullable = true)
 |-- brokerage_external_id: string (nullable = true)
 |-- brokerage_logo_url: string (nullable = true)
 |-- brokerage_name: string (nullable = true)
 |-- brokerage_phone: s

In [115]:
new_df = my_spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","backend_production")\
    .option("collection", "properties")\
    .schema(df.schema)\
    .load()

In [119]:
new_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: string (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- brokerage: string (nullable = true)
 |-- brokerage_email: string (nullable = true)
 |-- brokerage_external_id: string (nullable = true)
 |-- brokerage_logo_url: string (nullable = true)
 |-- brokerage_name: string (nullable = true)
 |-- brokerage_phone: s

In [150]:
new_df.registerTempTable("table")

In [151]:
#df_ca=sqlContext.sql("select * from table where country in ('Canada','CA')")

In [154]:
df_ca=sqlContext.sql(
'''
select 
utilities,                
parking_types,            
year_built,               
location_ids,            
lower_location_id,        
agent_fname,               
agent_lname,               
property_sub_type,        
listing_type,              
location,                  
postal,                    
bathrooms,                 
external_type,             
agent_key,                 
address_street,            
beds,                      
brokerage_external_id,     
brokerage_name,            
address_number,            
city,                      
country,                   
external_id,              
has_pool,                  
geocoded,                  
has_fireplace,             
has_garage,                
images,                    
latitude,                  
listing_status,            
longitude,                 
mls_number,                
price_cents,               
property_type,             
province,                  
status,                    
_id 
from table where country in ('Canada','CA')
'''
)


In [155]:
#df_ca.count()

82785

In [ ]:
# drop unecessary features (more than 50% records are null)
ca_drop_features=["has_pond", #100
"has_dock",
"foreclosure",              
"is_wired",                 
"has_barbecue_area",        
"is_new_construction",      
"intercom",                 
"has_basement",             
"heating_systems",          
"heating_fuels",            
"has_wet_bar",              
"has_vaulted_ceiling",      
"has_sprinkler_system",     
"has_sports_court",         
"has_skylight",             
"has_security_system",      
"has_sauna",                
"has_rv_parking",           
"has_porch",                
"has_ceiling_fan",         
"has_patio",                
"has_mother_in_law",        
"has_jetted_bath_tub",      
"has_hot_tub_spa",          
"has_green_house",          
"has_gated_entry",          
"has_garden",               
"has_deck",                
"has_disabled_access",      
"has_elevator",
"cooling_systems",#            100.00
"brokerage_website_url",#      100.00
"lead_routing_email",#         100.00
"architecture_style",#         100.00
"has_double_pane_windows",#    100.00
"agent_phone",#                100.00
"has_doorman",#                100.00
"brokerage_logo_url",#         100.00
"fees",#                       100.00
"external_url",#               100.00
"agent_id",#                   100.00
"num_floors",#                 100.00
"open_houses",#                100.00
"exterior_types",#             100.00
"price_cents_sqft",#           100.00
"brokerage_phone",#            100.00
"agent_email",#                100.00
"rooms",#                      100.00
"tax_amount",#                 100.00
"tags",#                       100.00
"agent_avatar_url",#           100.00
"scoring",#                    100.00
"school",#                     100.00
"brokerage_email",#            100.00
"room_count",#                 100.00
"building_sqft",#              99.33
"lot_sqft",#                   97.40
"total_sqft",#                 94.25
"roof_types",#                 86.29
"unit_number",#81.94
"building_utilities",#79.65
"floors",#65.05
"partial_bathrooms", # 54.05  
"is_waterfront"                  
]

In [156]:
df_ca.registerTempTable("ca_table")

In [157]:
df_ca.printSchema()

root
 |-- utilities: string (nullable = true)
 |-- parking_types: string (nullable = true)
 |-- year_built: string (nullable = true)
 |-- location_ids: string (nullable = true)
 |-- lower_location_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- property_sub_type: string (nullable = true)
 |-- listing_type: string (nullable = true)
 |-- location: string (nullable = true)
 |-- postal: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- external_type: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- brokerage_external_id: string (nullable = true)
 |-- brokerage_name: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- external_id: string (nullable = true)
 |-- has_pool: string (nullable = true)
 |-- 

In [158]:
#df_ca.select("utilities").distinct().count()

25018

In [161]:
sqlContext.sql("select utilities, count(*) from ca_table group by 1 order by count(*) desc ").show()

+--------------------+--------+
|           utilities|count(1)|
+--------------------+--------+
|                null|   22909|
|             ["All"]|    3885|
|["Washer", "Dryer...|    2713|
|["Refrigerator", ...|     993|
|["Dishwasher", "R...|     835|
|  ["Central Vacuum"]|     627|
|            ["None"]|     495|
|      ["Dishwasher"]|     474|
|["Washer", "Refri...|     414|
|["All", "Central ...|     391|
|     ["See remarks"]|     387|
|["Dryer - Electri...|     347|
|["Dryer - Electri...|     338|
|["Garage door ope...|     320|
|["Dishwasher", "D...|     281|
|["Dryer", "Dishwa...|     261|
|["Washer", "Refri...|     252|
|["Garage door ope...|     236|
|["Refrigerator", ...|     225|
|["Washer", "Dishw...|     206|
+--------------------+--------+
only showing top 20 rows



In [ ]:
sqlContext.sql("select utilities, count(*) from ca_table group by 1 order by count(*) desc ").show()

In [163]:
sqlContext.sql("select INSTR(utilities,'Washer') as water,  select INSTR(utilities,'Dryer') as Dryer from ca_table").show()

+------------------------+
|instr(utilities, Washer)|
+------------------------+
|                    null|
|                    null|
|                      45|
|                       0|
|                    null|
|                      21|
|                      67|
|                      35|
|                      57|
|                    null|
|                       0|
|                       0|
|                    null|
|                       0|
|                      35|
|                    null|
|                    null|
|                      21|
|                    null|
|                    null|
+------------------------+
only showing top 20 rows



In [ ]:
df_ca_drop=df_ca.drop(ca_drop_features).collect()

In [142]:
test=new_df.filter(new_df["country"]=="CA")

In [143]:
test.count()

18343

In [ ]:
from pyspark.sql import functions as F
new_df.filter(F.isnull("state")).show()

In [ ]:
df.where(col(k).isNull()).count()

In [134]:
new_df.where(new_df.state.isNull()).count()

2369118

In [145]:
test.state.describe().show()

TypeError: 'Column' object is not callable

In [148]:
# Count number of null rows in each column - very slow
from pyspark.sql.functions import isnan, when, count, col
aa=test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in test.columns])

In [149]:
aa.show()

+---+--------------+--------------+----------------+-----------+-----------------+-----------+--------+---------+-----------+----------+-----------+------------------+-----------------+--------+---------+----+---------+---------------+---------------------+------------------+--------------+---------------+---------------------+-------------+-------------+------------------+----+---------------+-------+----------------+----------+-----------+----------------+--------------+-----------+-------------+------------+-----+----------+------+-------------+-----------+----------+---------+------+--------+----------+---------+-----------------+------------+---------------+--------+-------------------+--------+-----------+-----------------------+------------+-------------+----------+----------+---------------+---------------+---------------+-------------------+-----------------+---------+--------+--------+---------+--------------+---------+-------------------+------------+----------------+-------

In [122]:
pandas_df=new_df.toPandas()
#df.filter(df(colName).isNull || df(colName) === "" || df(colName).isNaN).count()


Py4JJavaError: An error occurred while calling o8754.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 39 tasks (1037.4 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.appl

In [117]:
import pyspark.sql.functions as F

df_agg = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in new_df.columns])

In [121]:
new_df.info()

AttributeError: 'DataFrame' object has no attribute 'info'

In [118]:
df_agg.show()

Py4JJavaError: An error occurred while calling o9476.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost, executor driver): com.mongodb.spark.exceptions.MongoTypeConversionException: Cannot cast BOOLEAN into a NullType (value: BsonBoolean{value=true})
	at com.mongodb.spark.sql.MapFunctions$.com$mongodb$spark$sql$MapFunctions$$convertToDataType(MapFunctions.scala:200)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:39)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:37)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.coll

In [52]:
'''
df= my_spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","backend_production")\
    .option("collection", "properties")\
    .option("inferSchema", None)\
    .load()
.option("inferSchema", None)\
    .option(schema,data_df.schema)\
  '''  

In [ ]:
df.coalesce(1).write.format('json').save('/your_path/output_directory')

In [170]:
new_df.groupby("country").show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [175]:
new_df.registerTempTable("table")

In [185]:
df_ca=sqlContext.sql("select * from table where country = ('CA')")

In [186]:
df_ca.coalesce(1).write.format('json').save('df_ca.json')

In [176]:
sqlContext.sql("select country, count(*) from table group by country").show()

+-------+--------+
|country|count(1)|
+-------+--------+
|     MX|       1|
|   null|   16289|
|     CA|   18343|
|     IO|       1|
|     US| 2269170|
| Mexico|       8|
| Canada|   64442|
|     PR|     891|
+-------+--------+



In [174]:
new_df.select("country").distinct().show()

+-------+
|country|
+-------+
|     MX|
|   null|
|     CA|
|     IO|
|     US|
| Mexico|
| Canada|
|     PR|
+-------+



In [180]:
sqlContext.sql("select city,state, country, count(*) from table where country is null group by city, state, country order by count(*) desc").show()


+------------+-----+-------+--------+
|        city|state|country|count(1)|
+------------+-----+-------+--------+
|Indianapolis| null|   null|     834|
|      Boston| null|   null|     752|
|    Plymouth| null|   null|     257|
|  Evansville| null|   null|     221|
|      Muncie| null|   null|     201|
|  Barnstable| null|   null|     188|
|  Fort Wayne| null|   null|     176|
| Springfield| null|   null|     170|
|      Kokomo| null|   null|     168|
|      Marion| null|   null|     166|
|      Carmel| null|   null|     164|
|    Falmouth| null|   null|     148|
|   Westfield| null|   null|     145|
|     Fishers| null|   null|     141|
| New Bedford| null|   null|     140|
|    Anderson| null|   null|     132|
|      Newton| null|   null|     130|
|  Fall River| null|   null|     125|
|    Yarmouth| null|   null|     118|
|     Bedford| null|   null|     110|
+------------+-----+-------+--------+
only showing top 20 rows



In [161]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: boolean (nullable = true)
 |-- basement: boolean (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:

In [ ]:
##Following are the Schema type mismatches when Spark loads property data

'total_sqft' - Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'building_sqft' - Cannot cast STRING into a DoubleType (value: BsonString{value=''})
'lot_sqft' - Cannot cast STRING into a DoubleType (value: BsonString{value='NA'})
'external_url'- Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'furnished' - Cannot cast BOOLEAN into a NullType (value: BsonBoolean{value=false})
'main_level_sqft'  - Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'open_house_count' - Cannot cast STRING into a NullType (value: BsonString{value=''})    
'state' - Cannot cast STRING into a NullType (value: BsonString{value='active'})
'stories' - Cannot cast STRING into a IntegerType (value: BsonString{value=''})
'total_sqft'- Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'available_showing' -  Cannot cast BOOLEAN into a NullType 


In [143]:
from pyspark.sql.types import IntegerType
data_df = df.withColumn("state", df["state"].cast(StringType()))
data_df1 = data_df.withColumn("stories", df["stories"].cast(StringType()))
data_df = data_df1.withColumn("total_sqft", df["total_sqft"].cast(StringType()))
data_df = data_df.withColumn("building_sqft", df["building_sqft"].cast(StringType()))
data_df = data_df.withColumn("lot_sqft", df["lot_sqft"].cast(StringType()))
data_df = data_df.withColumn("external_url", df["external_url"].cast(StringType()))
data_df = data_df.withColumn("furnished", df["furnished"].cast(StringType()))
data_df = data_df.withColumn("main_level_sqft", df["main_level_sqft"].cast(StringType()))
data_df = data_df.withColumn("open_house_count", df["open_house_count"].cast(StringType()))
data_df = data_df.withColumn("total_sqft", df["total_sqft"].cast(StringType()))
data_df = data_df.withColumn("available_showing", df["available_showing"].cast(StringType()))
data_df = data_df.withColumn("agent_avatar_url", df["agent_avatar_url"].cast(StringType()))
data_df = data_df.withColumn("basement", df["basement"].cast(StringType()))
data_df = data_df.withColumn("bathrooms", df["bathrooms"].cast(StringType()))
data_df = data_df.withColumn("floors_number", df["floors_number"].cast(StringType()))
data_df = data_df.withColumn("foreclosure", df["foreclosure"].cast(StringType()))
data_df = data_df.withColumn("garage", df["garage"].cast(StringType()))
data_df = data_df.withColumn("geocoded", df["geocoded"].cast(StringType()))
data_df = data_df.withColumn("num_floors", df["num_floors"].cast(StringType()))
data_df = data_df.withColumn("open_house_count", df["open_house_count"].cast(StringType()))
data_df = data_df.withColumn("scoring", df["scoring"].cast(StringType()))
data_df = data_df.withColumn("showing_date", df["showing_date"].cast(StringType()))





In [162]:
for i in df.columns:
    df= df.withColumn(i, df[i].cast(StringType()))

In [163]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: string (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- brokerage: string (nullable = true)
 |-- brokerage_email: string (nullable = true)
 |-- brokerage_external_id: string (nullable = true)
 |-- brokerage_logo_url: string (nullable = true)
 |-- brokerage_name: string (nullable = true)
 |-- brokerage_phone: s

In [156]:
data_df.select("state").show()

Py4JJavaError: An error occurred while calling o22464.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 31, localhost, executor driver): com.mongodb.spark.exceptions.MongoTypeConversionException: Cannot cast STRING into a NullType (value: BsonString{value='active'})
	at com.mongodb.spark.sql.MapFunctions$.com$mongodb$spark$sql$MapFunctions$$convertToDataType(MapFunctions.scala:200)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:39)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:37)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scal

In [144]:
data_df1.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: null (nullable = true)
 |-- basement: boolean (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:Ful

In [126]:
for i in df.columns:
    data_df = df.withColumn(i, df[i].cast(StringType()))

In [127]:
data_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: null (nullable = true)
 |-- basement: boolean (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:Ful

In [130]:
new_df.show()

Py4JJavaError: An error occurred while calling o21383.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 29, localhost, executor driver): com.mongodb.spark.exceptions.MongoTypeConversionException: Cannot cast BOOLEAN into a NullType (value: BsonBoolean{value=true})
	at com.mongodb.spark.sql.MapFunctions$.com$mongodb$spark$sql$MapFunctions$$convertToDataType(MapFunctions.scala:200)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:39)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:37)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.

In [132]:
new_df.schema

StructType(List(StructField(_id,StructType(List(StructField(oid,StringType,true))),true),StructField(address_number,StringType,true),StructField(address_street,StringType,true),StructField(agent_avatar_url,NullType,true),StructField(agent_email,StringType,true),StructField(agent_external_id,StringType,true),StructField(agent_fname,StringType,true),StructField(agent_id,StringType,true),StructField(agent_key,StringType,true),StructField(agent_lname,StringType,true),StructField(agent_name,StringType,true),StructField(agent_phone,StringType,true),StructField(architecture_style,StringType,true),StructField(available_showing,NullType,true),StructField(basement,BooleanType,true),StructField(bathrooms,DoubleType,true),StructField(beds,IntegerType,true),StructField(brokerage,StructType(List(StructField(address,StructType(List(StructField(commons:City,StringType,true),StructField(commons:Country,StringType,true),StructField(commons:FullStreetAddress,StringType,true),StructField(commons:PostalCod

In [114]:
data_df = df.withColumn("basement", df["basement"].cast(StringType()))

In [118]:
data_df.select("basement")

DataFrame[basement: boolean]

In [113]:
for i in df.columns:
    print(i)

_id
address_number
address_street
agent_avatar_url
agent_email
agent_external_id
agent_fname
agent_id
agent_key
agent_lname
agent_name
agent_phone
architecture_style
available_showing
basement
bathrooms
beds
brokerage
brokerage_email
brokerage_external_id
brokerage_logo_url
brokerage_name
brokerage_phone
brokerage_website_url
building_sqft
building_type
building_utilities
city
cooling_systems
country
cover_public_url
created_at
description
disclose_address
exterior_types
external_id
external_type
external_url
fees
fireplaces
floors
floors_number
foreclosure
full_baths
furnished
garage
geocoded
half_baths
has_attic
has_barbecue_area
has_basement
has_ceiling_fan
has_deck
has_disabled_access
has_dock
has_doorman
has_double_pane_windows
has_elevator
has_fireplace
has_garage
has_garden
has_gated_entry
has_green_house
has_hot_tub_spa
has_jetted_bath_tub
has_mother_in_law
has_patio
has_pond
has_pool
has_porch
has_rv_parking
has_sauna
has_security_system
has_skylight
has_sports_court
has_sprin

In [116]:
data_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: null (nullable = true)
 |-- basement: string (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:Full

In [105]:
data_df = df.withColumn(  ["state", df["state"].cast(StringType())] )



TypeError: withColumn() missing 1 required positional argument: 'col'

In [65]:
data_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: null (nullable = true)
 |-- basement: boolean (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:Ful

SyntaxError: invalid syntax (<ipython-input-91-8c49985b239e>, line 1)

In [92]:
df01.select("available_showing").show()



Py4JJavaError: An error occurred while calling o922.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 1 times, most recent failure: Lost task 0.0 in stage 23.0 (TID 23, localhost, executor driver): com.mongodb.spark.exceptions.MongoTypeConversionException: Cannot cast BOOLEAN into a NullType (value: BsonBoolean{value=true})
	at com.mongodb.spark.sql.MapFunctions$.com$mongodb$spark$sql$MapFunctions$$convertToDataType(MapFunctions.scala:200)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:39)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:37)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.co

In [41]:
schemaa=df01.schema


In [57]:
from pyspark.sql.types import *


In [58]:
schemaa= StructType(fields=data_schema)

AttributeError: 'str' object has no attribute 'name'

In [12]:
df = df01.withColumn('state', F.col('Total').cast(types.DecimalType()))


In [16]:
df01.registerTempTable("table")

In [21]:
df01.select("state").cast(state).as("string").show()

SyntaxError: invalid syntax (<ipython-input-21-158eb0902f37>, line 1)

In [22]:
df01.select(df01.state.cast("string").alias('statesss')).collect()

Py4JJavaError: An error occurred while calling o259.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 780, localhost, executor driver): com.mongodb.spark.exceptions.MongoTypeConversionException: Cannot cast STRING into a NullType (value: BsonString{value='active'})
	at com.mongodb.spark.sql.MapFunctions$.com$mongodb$spark$sql$MapFunctions$$convertToDataType(MapFunctions.scala:200)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:39)
	at com.mongodb.spark.sql.MapFunctions$$anonfun$3.apply(MapFunctions.scala:37)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at 

In [13]:
df_id.count()

TypeError: count() takes exactly one argument (0 given)

In [3]:
client = MongoClient()
#client = MongoClient()
db = client.backend_production

In [10]:
dd_us=db.properties.find( {"country": "US"}).limit(100000)

In [11]:
df = pd.DataFrame.from_dict(dd_us, orient='columns')

In [12]:
df.head()

,_id,address_number,address_street,agent_avatar_url,agent_email,agent_external_id,agent_fname,agent_id,agent_key,agent_lname,...,tags,tax_amount,three_quarter_bathrooms,total_sqft,unit_number,updated_at,utilities,water,water_heater,year_built
0,5a3ceff1aed0f133b9456d98,2501,Avery Street,NaN,stephen@chalkandgibbs.com,NaN,Stephen,20151206221506028181000000,3yd-CCMLSNC-20151206221506028181000000,W. Chalk Jr.,...,None,None,0.0,NaN,None,2019-04-06 08:42:45.705,None,NaN,NaN,1973.0
1,5a3ceff9aed0f133b9456f3c,19,Privateer Drive,NaN,wheland@hargray.com,NaN,Tom,wheland,3yd-BCARSC-wheland,Wheland,...,None,None,0.0,NaN,None,2019-04-06 08:42:45.773,None,NaN,NaN,1986.0
2,5a3cf088aed0f133b945837f,1701,Salter Path Road,NaN,carolyn@coastland.com,NaN,Carolyn,20151206221518063769000000,3yd-CCMLSNC-20151206221518063769000000,Wood,...,None,None,0.0,NaN,I204,2019-04-07 20:44:22.938,"[Dishwasher, Dryer, Microwave, Oven, Refrigera...",NaN,NaN,2003.0
3,5a3cf08aaed0f133b9458575,203,Bluewater Cove,NaN,roy@coastland.com,NaN,Roy,20151206221451745345000000,3yd-CCMLSNC-20151206221451745345000000,Parker,...,None,None,0.0,NaN,None,2019-04-06 08:42:45.951,None,NaN,NaN,2018.0
4,5a3cf0abaed0f133b9458be6,11,Shults Road,NaN,jimlivingstonrealestate@gmail.com,NaN,James,livingstonj,3yd-BCARSC-livingstonj,R. Livingston,...,None,None,0.0,NaN,None,2019-04-06 08:42:46.032,None,NaN,NaN,1957.0


In [13]:
df.to_csv('data_notebook.csv')




In [18]:
df.select('_id','address_number','address_street','agent_avatar_url','agent_email','agent_external_id','agent_fname','agent_id','agent_key','agent_lname','agent_name','agent_phone','architecture_style','available_showing','basement','bathrooms','beds','brokerage','brokerage_email','brokerage_external_id','brokerage_logo_url','brokerage_name','brokerage_phone','brokerage_website_url','building_type','building_utilities','city','cooling_systems','country','cover_public_url','created_at','description','disclose_address','exterior_types','external_id','external_url','fees','fireplaces','floors','floors_number','foreclosure','full_baths','garage','geocoded','half_baths','has_attic','has_barbecue_area','has_basement','has_ceiling_fan','has_deck','has_disabled_access','has_dock','has_doorman','has_double_pane_windows','has_elevator','has_fireplace','has_garage','has_garden','has_gated_entry','has_green_house','has_hot_tub_spa','has_jetted_bath_tub','has_mother_in_law','has_patio','has_pond','has_pool','has_porch','has_rv_parking','has_sauna','has_security_system','has_skylight','has_sports_court','has_sprinkler_system','has_vaulted_ceiling','has_wet_bar','heating_fuels','heating_systems','images','intercom','is_cable_ready','is_new_construction','is_waterfront','is_wired','latitude','lead_routing_email','listing_category','listing_date','listing_status','listing_title','listing_type','listing_url','living_area','location','location_ids','longitude','lower_location_id','mls_id','mls_name','mls_number','modified_at','num_floors','one_quarter_bathrooms','parking','parking_types','partial_bathrooms','participant','patio','pool','postal','price_cents','price_cents_sqft','property_sub_type','property_type','provider_category','provider_name','provider_url','province','roof','roof_types','room_count','rooms','school','scoring','showing_date','status','tags','tax_amount','three_quarter_bathrooms','unit_number','updated_at','utilities','water','water_heater','year_built').show()
    
    

TypeError: select() takes from 2 to 3 positional arguments but 135 were given

In [16]:
df['total_sqft']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
         ..
99970   NaN
99971   NaN
99972   NaN
99973   NaN
99974   NaN
99975   NaN
99976   NaN
99977   NaN
99978   NaN
99979   NaN
99980   NaN
99981   NaN
99982   NaN
99983   NaN
99984   NaN
99985   NaN
99986   NaN
99987   NaN
99988   NaN
99989   NaN
99990   NaN
99991   NaN
99992   NaN
99993   NaN
99994   NaN
99995   NaN
99996   NaN
99997   NaN
99998   NaN
99999   NaN
Name: total_sqft, Length: 100000, dtype: float64

In [ ]:
schema='root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address_number: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- agent_avatar_url: null (nullable = true)
 |-- agent_email: string (nullable = true)
 |-- agent_external_id: string (nullable = true)
 |-- agent_fname: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- agent_key: string (nullable = true)
 |-- agent_lname: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)
 |-- architecture_style: string (nullable = true)
 |-- available_showing: boolean (nullable = true)
 |-- basement: boolean (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- beds: integer (nullable = true)
 |-- brokerage: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- commons:City: string (nullable = true)
 |    |    |-- commons:Country: string (nullable = true)
 |    |    |-- commons:FullStreetAddress: string (nullable = true)
 |    |    |-- commons:PostalCode: string (nullable = true)
 |    |    |-- commons:StateOrProvince: string (nullable = true)
 |    |    |-- commons:UnitNumber: string (nullable = true)
 |    |    |-- commons:address-preference-order: string (nullable = true)
 |    |    |-- commons:preference-order: string (nullable = true)
 |    |-- logo_url: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phone: string (nullable = true)
 |    |-- website_url: string (nullable = true)
 |-- brokerage_email: string (nullable = true)
 |-- brokerage_external_id: string (nullable = true)
 |-- brokerage_logo_url: string (nullable = true)
 |-- brokerage_name: string (nullable = true)
 |-- brokerage_phone: string (nullable = true)
 |-- brokerage_website_url: string (nullable = true)
 |-- building_sqft: double (nullable = true)
 |-- building_type: string (nullable = true)
 |-- building_utilities: string (nullable = true)
 |-- city: string (nullable = true)
 |-- cooling_systems: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- country: string (nullable = true)
 |-- cover_public_url: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- description: string (nullable = true)
 |-- disclose_address: string (nullable = true)
 |-- exterior_types: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- external_id: string (nullable = true)
 |-- external_type: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |-- external_url: string (nullable = true)
 |-- fees: null (nullable = true)
 |-- fireplaces: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- floors_number: double (nullable = true)
 |-- foreclosure: boolean (nullable = true)
 |-- full_baths: integer (nullable = true)
 |-- furnished: null (nullable = true)
 |-- garage: boolean (nullable = true)
 |-- geocoded: boolean (nullable = true)
 |-- half_baths: integer (nullable = true)
 |-- has_attic: null (nullable = true)
 |-- has_barbecue_area: boolean (nullable = true)
 |-- has_basement: boolean (nullable = true)
 |-- has_ceiling_fan: boolean (nullable = true)
 |-- has_deck: boolean (nullable = true)
 |-- has_disabled_access: boolean (nullable = true)
 |-- has_dock: boolean (nullable = true)
 |-- has_doorman: boolean (nullable = true)
 |-- has_double_pane_windows: boolean (nullable = true)
 |-- has_elevator: boolean (nullable = true)
 |-- has_fireplace: boolean (nullable = true)
 |-- has_garage: boolean (nullable = true)
 |-- has_garden: boolean (nullable = true)
 |-- has_gated_entry: boolean (nullable = true)
 |-- has_green_house: boolean (nullable = true)
 |-- has_hot_tub_spa: boolean (nullable = true)
 |-- has_jetted_bath_tub: boolean (nullable = true)
 |-- has_mother_in_law: boolean (nullable = true)
 |-- has_patio: boolean (nullable = true)
 |-- has_pond: boolean (nullable = true)
 |-- has_pool: boolean (nullable = true)
 |-- has_porch: boolean (nullable = true)
 |-- has_rv_parking: boolean (nullable = true)
 |-- has_sauna: boolean (nullable = true)
 |-- has_security_system: boolean (nullable = true)
 |-- has_skylight: boolean (nullable = true)
 |-- has_sports_court: boolean (nullable = true)
 |-- has_sprinkler_system: boolean (nullable = true)
 |-- has_vaulted_ceiling: boolean (nullable = true)
 |-- has_wet_bar: boolean (nullable = true)
 |-- heating_fuels: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- heating_systems: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: struct (nullable = true)
 |    |    |    |-- oid: string (nullable = true)
 |    |    |-- created_at: timestamp (nullable = true)
 |    |    |-- modified_at: timestamp (nullable = true)
 |    |    |-- original_url: string (nullable = true)
 |    |    |-- sha1: string (nullable = true)
 |    |    |-- updated_at: timestamp (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- intercom: boolean (nullable = true)
 |-- is_cable_ready: boolean (nullable = true)
 |-- is_new_construction: boolean (nullable = true)
 |-- is_waterfront: boolean (nullable = true)
 |-- is_wired: boolean (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lead_routing_email: string (nullable = true)
 |-- listing_category: string (nullable = true)
 |-- listing_date: timestamp (nullable = true)
 |-- listing_status: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |-- listing_title: string (nullable = true)
 |-- listing_type: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- living_area: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- location_ids: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- oid: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- lot_sqft: double (nullable = true)
 |-- lower_location_id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- main_level_sqft: double (nullable = true)
 |-- mls_id: string (nullable = true)
 |-- mls_name: string (nullable = true)
 |-- mls_number: string (nullable = true)
 |-- modified_at: timestamp (nullable = true)
 |-- num_floors: double (nullable = true)
 |-- one_quarter_bathrooms: integer (nullable = true)
 |-- open_house_count: null (nullable = true)
 |-- parking: string (nullable = true)
 |-- parking_types: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- partial_bathrooms: integer (nullable = true)
 |-- participant: struct (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- key: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- office_phone: string (nullable = true)
 |    |-- primary_contact_phone: string (nullable = true)
 |    |-- website_url: string (nullable = true)
 |-- patio: boolean (nullable = true)
 |-- pool: boolean (nullable = true)
 |-- postal: string (nullable = true)
 |-- price_cents: integer (nullable = true)
 |-- price_cents_sqft: integer (nullable = true)
 |-- property_sub_type: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- provider_category: string (nullable = true)
 |-- provider_name: string (nullable = true)
 |-- provider_url: string (nullable = true)
 |-- province: string (nullable = true)
 |-- roof: string (nullable = true)
 |-- roof_types: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- room_count: integer (nullable = true)
 |-- rooms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- school: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- scoring: null (nullable = true)
 |-- showing_date: timestamp (nullable = true)
 |-- state: null (nullable = true)
 |-- status: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |-- stories: null (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- symbol: string (nullable = true)
 |-- tax_amount: string (nullable = true)
 |-- three_quarter_bathrooms: integer (nullable = true)
 |-- total_sqft: double (nullable = true)
 |-- unit_number: string (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- utilities: string (nullable = true)
 |-- water: string (nullable = true)
 |-- water_heater: string (nullable = true)

    |-- year_built: integer (nullable = true)'

In [83]:
##Following are the Schema type mismatches when Spark loads property data

'total_sqft' - Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'building_sqft' - Cannot cast STRING into a DoubleType (value: BsonString{value=''})
'lot_sqft' - Cannot cast STRING into a DoubleType (value: BsonString{value='NA'})
'external_url'- Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'furnished' - Cannot cast BOOLEAN into a NullType (value: BsonBoolean{value=false})
'main_level_sqft'  - Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'open_house_count' - Cannot cast STRING into a NullType (value: BsonString{value=''})    
'state' - Cannot cast STRING into a NullType (value: BsonString{value='active'})
'stories' - Cannot cast STRING into a IntegerType (value: BsonString{value=''})
'total_sqft'- Cannot cast STRING into a DoubleType (value: BsonString{value='1398'})
'available_showing' -     

SyntaxError: invalid syntax (<ipython-input-83-b89e53631f67>, line 4)